In [21]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from imgaug import augmenters as iaa
from sklearn.ensemble import IsolationForest
import xgboost as xgb
import pandas as pd
import cv2
import warnings
import json
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Load the dataset from the CSV file
data = pd.read_csv("../csv/minority_train.csv")

# Initialize variables to store sampled data as lists
sampled_image_paths = []
sampled_class_labels = []

# Specify the number of samples per class and the total number of samples
samples_per_class = 20  # Adjust as needed
total_samples = 100

# Iterate through unique classes
unique_classes = data["dx"].unique()
for class_label in unique_classes:
    # Select samples for the current class
    class_data = data[data["dx"] == class_label].head(samples_per_class)
    
    # Append the sampled image paths and class labels to the result lists
    sampled_image_paths.extend(class_data["image_pth"].tolist())
    sampled_class_labels.extend(class_data["dx"].tolist())


# Initialize an empty list to store images
images = []

# Load images using OpenCV and convert to grayscale
for image_path in sampled_image_paths:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    images.append(image)


X = np.array(images)  # Convert to numpy array

# Convert class labels to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_class_labels)

# Flatten the image data
X = X.reshape(X.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"ytest: {np.unique(y_test)}")
print(f"ytrain: {np.unique(y_train)}")

class_distribution_test = np.bincount(y_test)
class_distribution_train = np.bincount(y_train)
print("Class distribution in y_test:", class_distribution_test)
print("Class distribution in y_train:", class_distribution_train)

# Generate a synthetic multiclass classification dataset (you should replace this with your real data)
# X, y = make_classification(n_samples=1000, n_features=20, n_classes=5, n_informative=5, random_state=42)


# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = (X_train * 255).astype(np.uint8)
X_test = (X_test * 255).astype(np.uint8)

# Define a list of augmentation techniques
augmentations = [
    iaa.Fliplr(0.5),  # Horizontal flip with 50% probability
    iaa.Flipud(0.5),  # Vertical flip with 50% probability
    # iaa.Affine(rotate=(-45, 45)),  # Rotation between -45 and 45 degrees
    iaa.Multiply((0.5, 1.5)),  # Brightness multiplication between 0.5 and 1.5
]

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=1)

# Initialize AdaBoostClassifier with the base classifier

adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)

# Create a dictionary to store classification reports for each augmentation and class
augmentation_reports = {}
unique_classes_test = np.unique(y_test)


# Train AdaBoost with each augmentation technique for each class and record classification report
num_iterations = 10  # Number of iterations to test each augmentation
consolidated_x = []
consolidated_y = []
for augmentation in augmentations:
    augmentation_reports[augmentation] = {}
    print(f"Augmentation: {augmentation.name}")
    for class_label in np.unique(y_train):
        print(f"Class: {class_label}")
        class_mask = (y_train == class_label)
        other_mask = (y_train != class_label)
        
        other_labels = []
        class_labels = []
        
        for label in other_mask:
            other_labels.append(label)
        
        for label in class_mask:
            class_labels.append(label)
        
        augmented_X_train = X_train[class_mask].copy()
        non_augmented_X_train = X_train[other_mask].copy()
        
        # Apply augmentation to the samples of the current class
        augmented_X_train = augmentation.augment_images(augmented_X_train)
        
        
        # augmented should be combined with X-train left over data.
        all_data = np.concatenate((augmented_X_train, non_augmented_X_train), axis=0)
        all_labels = np.concatenate((y_train[class_mask], y_train[other_mask]), axis=0)
        
        # Fit AdaBoost classifier for the current class
        # adaboost_classifier.fit(augmented_X_train, y_train[class_mask])
        adaboost_classifier.fit(all_data, all_labels)
        
        
        # Evaluate the classifier on the test set
        y_pred = adaboost_classifier.predict(X_test)
        # Check unique classes in y_test and y_pred
        print(f'Unique classes test : {np.unique(y_test)}')
        print(f'Unique classes pred : {np.unique(y_pred)}')
        
        # # Confirm that both sets of unique classes match
        # if np.array_equal(unique_classes_test, unique_classes_pred):
        #     target_names = [f'Class_{i}' for i in unique_classes_test]
        # else:
        #     print("Mismatch in unique classes between y_test and y_pred.")
        
        # Then, when calling classification_report, pass the target_names parameter
        classification_rep = classification_report(y_test, y_pred, target_names=unique_classes_test, output_dict=True)
        # Store the classification report in the dictionary with keys based on augmentation and class
        augmentation_reports[augmentation][f'Class_{class_label}'] = classification_rep

# Print the classification report for each augmentation technique and class
for augmentation, class_reports in augmentation_reports.items():
    print("#" * 40)
    print(f"\nAugmentation:  == {augmentation.name} ==\n")
    print("=" * 40)
    for class_label, report in class_reports.items():
        print(f"Class {class_label} Classification Report:")
        for metric, value in report.items():
            if metric != 'accuracy' and f"Class_{metric}" == class_label:
                print(f"metric: Class_{metric}, class_label: {class_label}")
                print(f"{metric}: {value:.2f}" if isinstance(value, (float, np.float32)) else f"{metric}: {value}")
        print("=" * 40)
    print("#" * 40)


with open('./augment/metrics.txt', 'w+')as metrics:
    json.dump(augmentation_reports, metrics)
# print("*" * 40)
# print(class_reports)
# print("*" * 40)



ytest: [0 1 2 3 4 5]
ytrain: [0 1 2 3 4 5]
Class distribution in y_test: [3 5 3 3 5 5]
Class distribution in y_train: [17 15 17 17 15 15]
Augmentation: UnnamedFliplr
Class: 0
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 1
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 5]
Class: 2
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 3
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 4
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 2 5]
Class: 5
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 2]
Augmentation: UnnamedFlipud
Class: 0
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 1
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 2
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 3
Unique classes test : [0 1 2 3 4 5]
Unique classes pred : [0 1 5]
Class: 4
Unique classes test : [0 1 2 3 4 5]
Unique classe

In [15]:
y_test_transformed = y_test
y_train_transformed = y_train
label = 0

selected_ytest = (y_test_transformed == label)
selected_ytrain = (y_train_transformed == label)

notSelected_ytest = (y_test_transformed != label)
notSelected_ytrain = (y_train_transformed != label)


print(y_test[selected_ytest])
print(y_test[notSelected_ytest])



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3 2 2 3 1 3 4 5 4 3 1 3 5 2 5 5 2 5 2 4 2 1 3 3 1 3 4 2 1 1 1 3 5 1 4 4 5
 2 4 3 4 3 1 1 4 1 1 3 3 3 2 2 5 5 2 1 4 5 1 2 3 4 1 4 5 4 2 5 1 1 2 5 1 1
 1 4 4 1 3 3 4 5 2 2 4 3 4 5 3 1 5 2 5 3 5 1 2 2 2 5 4 5 5]


In [19]:
# Change the labels to 0 for augmented class and 1 for other class
labels = []
for item in notSelected_ytrain:
    if item == 1:
        labels.append(1)        
    else:
        item = 0
        labels.append(item)

print(labels)


[1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 

In [14]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from imgaug import augmenters as iaa
from sklearn.ensemble import IsolationForest
import xgboost as xgb
import pandas as pd
import cv2
import warnings
import json
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Load the dataset from the CSV file
data = pd.read_csv("../csv/minority_train.csv")

# Initialize variables to store sampled data as lists
sampled_image_paths = []
sampled_class_labels = []

# Specify the number of samples per class and the total number of samples
samples_per_class = 100  # Adjust as needed
total_samples = 600

# Iterate through unique classes
unique_classes = data["dx"].unique()
for class_label in unique_classes:
    # Select samples for the current class
    class_data = data[data["dx"] == class_label].head(samples_per_class)
    
    # Append the sampled image paths and class labels to the result lists
    sampled_image_paths.extend(class_data["image_pth"].tolist())
    sampled_class_labels.extend(class_data["dx"].tolist())


# Initialize an empty list to store images
images = []

# Load images using OpenCV and convert to grayscale
for image_path in sampled_image_paths:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    images.append(image)


X = np.array(images)  # Convert to numpy array

# Convert class labels to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_class_labels)

# Flatten the image data
X = X.reshape(X.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"ytest: {np.unique(y_test)}")
print(f"ytrain: {np.unique(y_train)}")

class_distribution_test = np.bincount(y_test)
class_distribution_train = np.bincount(y_train)
print("Class distribution in y_test:", class_distribution_test)
print("Class distribution in y_train:", class_distribution_train)

# Generate a synthetic multiclass classification dataset (you should replace this with your real data)
# X, y = make_classification(n_samples=1000, n_features=20, n_classes=5, n_informative=5, random_state=42)


# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = (X_train * 255).astype(np.uint8)
X_test = (X_test * 255).astype(np.uint8)

# Define a list of augmentation techniques
augmentations = [
    iaa.Multiply((0.5, 1.5)),  # Brightness multiplication between 0.5 and 1.5
    iaa.Flipud(0.5),  # Vertical flip with 50% probability
    # iaa.Affine(rotate=(-45, 45)),  # Rotation between -45 and 45 degrees
    iaa.Fliplr(0.5),  # Horizontal flip with 50% probability
]

# Define the base classifier (Decision Tree)
base_classifier = DecisionTreeClassifier(max_depth=1)

# Initialize AdaBoostClassifier with the base classifier

adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)

# Create a dictionary to store classification reports for each augmentation and class
augmentation_reports = {}
unique_classes_test = np.unique(y_test)


# Train AdaBoost with each augmentation technique for each class and record classification report
num_iterations = 10  # Number of iterations to test each augmentation
consolidated_x = []
consolidated_y = []

for class_label in np.unique(y_train):
    print(f"Class: {class_label}")
    class_mask = (y_train == class_label)
    other_mask = (y_train != class_label)
    augmentation_reports[f'Class_{class_label}'] = {}
    for augmentation in augmentations:
        print(f'\tAugmentation: {augmentation.name}')
        # augmentation_reports[f'Class_{class_label}'] = {}    
        augmented_X_train = X_train[class_mask].copy()
        non_augmented_X_train = X_train[other_mask].copy()
    
        # Apply augmentation to the samples of the current class
        augmented_X_train = augmentation.augment_images(augmented_X_train)
    
        # augmented should be combined with X-train left over data.
        all_data = np.concatenate((augmented_X_train, non_augmented_X_train), axis=0)
        all_labels = np.concatenate((y_train[class_mask], y_train[other_mask]), axis=0)
    
        # Fit AdaBoost classifier for the current class
        # adaboost_classifier.fit(augmented_X_train, y_train[class_mask])
        adaboost_classifier.fit(all_data, all_labels)
    
    
        # Evaluate the classifier on the test set
        y_pred = adaboost_classifier.predict(X_test)
        # Check unique classes in y_test and y_pred
        print(f'\tClasses test : {y_test}')
        print(f'\tClasses pred : {y_pred}')
        
        # for idx in range(0, len(X_test)):
        #     cv2.imwrite(f'./images/{y_test[idx]},{y_pred[idx]}.jpg', X_test[idx].reshape(450, 600))
        # Then, when calling classification_report, pass the target_names parameter
        classification_rep = classification_report(y_test, y_pred, target_names=unique_classes_test, output_dict=True)
        print(f'\tClassification report: {classification_rep}')
        # Store the classification report in the dictionary with keys based on augmentation and class
        augmentation_reports[f'Class_{class_label}'][augmentation.name] = classification_rep

# Print the classification report for each augmentation technique and class
for class_report, augmentation in augmentation_reports.items():
    print("#" * 40)
    print(f"\Class:  == {class_report} ==\n")
    print("=" * 40)
    for augment, report in augmentation.items():
        print(f"Augmentation {augment} Classification Report:")
        for metric, value in report.items():
            if metric != 'accuracy':
                # print(f"metric: {metric}, value: {value}")
                print(f"{metric}: {value:.2f}" if isinstance(value, (float, np.float32)) else f"{metric}: {value}")
        print("=" * 40)
    print("#" * 40)


with open('metrics_100_per_class.txt', 'w+')as metrics:
   metrics.write(str(augmentation_reports))
# print("*" * 40)
# print(class_reports)
# print("*" * 40)

ytest: [0 1 2 3 4 5]
ytrain: [0 1 2 3 4 5]
Class distribution in y_test: [16 23 20 20 19 21]
Class distribution in y_train: [84 77 80 72 81 79]
Class: 0
 === Augmentation: UnnamedMultiply === 
Classes test : [0 3 2 2 3 1 3 4 5 4 3 1 3 0 5 2 5 5 2 5 2 4 2 1 3 0 3 1 3 4 2 1 1 1 3 5 1
 0 4 0 4 5 2 4 3 4 0 3 1 0 1 4 1 0 1 3 3 3 2 2 0 5 5 2 1 4 5 1 0 2 3 4 1 4
 5 4 2 5 0 1 1 2 5 0 1 1 1 4 4 1 3 3 4 0 5 2 2 0 4 3 4 5 3 1 5 2 0 5 3 5 1
 2 2 2 0 5 4 5 5]
Classes pred : [1 4 5 2 0 2 0 2 5 2 2 3 0 1 5 5 4 0 0 5 1 2 2 2 0 3 3 0 0 4 3 5 0 0 0 4 1
 5 4 1 0 4 1 0 0 2 5 0 1 0 0 4 0 2 0 0 0 5 2 0 0 2 2 5 5 2 2 0 0 3 0 5 5 4
 5 5 4 2 5 0 0 4 5 0 0 0 0 4 0 1 2 0 4 5 2 4 2 0 5 5 5 4 0 1 3 0 0 2 1 3 1
 5 5 1 1 2 5 4 4]
 === Augmentation: UnnamedFlipud === 
Classes test : [0 3 2 2 3 1 3 4 5 4 3 1 3 0 5 2 5 5 2 5 2 4 2 1 3 0 3 1 3 4 2 1 1 1 3 5 1
 0 4 0 4 5 2 4 3 4 0 3 1 0 1 4 1 0 1 3 3 3 2 2 0 5 5 2 1 4 5 1 0 2 3 4 1 4
 5 4 2 5 0 1 1 2 5 0 1 1 1 4 4 1 3 3 4 0 5 2 2 0 4 3 4 5 3 1 5 2 0 5 3 5 1
 2 2 2 0 5 4 

In [11]:
for class_report, augmentation in augmentation_reports.items():
    print("#" * 40)
    print(f"\Class:  == {class_report} ==\n")
    print("=" * 40)
    for augment, report in augmentation.items():
        print(f"Augmentation {augment} Classification Report:")
        for metric, value in report.items():
            if metric != 'accuracy':
                # print(f"metric: {metric}, value: {value}")
                print(f"{metric}: {value:.2f}" if isinstance(value, (float, np.float32)) else f"{metric}: {value}")
        print("=" * 40)
    print("#" * 40)

########################################
\Class:  == Class_0 ==

Augmentation UnnamedFliplr Classification Report:
0: {'precision': 0.16666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.26666666666666666, 'support': 3}
1: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}
2: {'precision': 0.2, 'recall': 0.6666666666666666, 'f1-score': 0.30769230769230765, 'support': 3}
3: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}
4: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}
5: {'precision': 0.5, 'recall': 0.2, 'f1-score': 0.28571428571428575, 'support': 5}
macro avg: {'precision': 0.14444444444444446, 'recall': 0.25555555555555554, 'f1-score': 0.14334554334554336, 'support': 24}
weighted avg: {'precision': 0.15, 'recall': 0.20833333333333334, 'f1-score': 0.13131868131868132, 'support': 24}
Augmentation UnnamedFlipud Classification Report:
0: {'precision': 0.18181818181818182, 'recall': 0.6666666666666666, 'f1-score': 0.2857142857142

In [12]:
with open('metrics.txt', 'w+')as metrics:
   metrics.write(str(augmentation_reports))

In [ ]:
# Load the dataset from the CSV file
data = pd.read_csv("../csv/minority_train.csv")

sampled_image_paths = []
sampled_class_labels = []
samples_per_class = 100  # Adjust as needed
total_samples = 600

unique_classes = data["dx"].unique()
for class_label in unique_classes:
    class_data = data[data["dx"] == class_label].head(samples_per_class)
    sampled_image_paths.extend(class_data["image_pth"].tolist())
    sampled_class_labels.extend(class_data["dx"].tolist())
images = []

for image_path in sampled_image_paths:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    images.append(image)

X = np.array(images)  # Convert to numpy array
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_class_labels)

X = X.reshape(X.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"ytest: {np.unique(y_test)}")
print(f"ytrain: {np.unique(y_train)}")

class_distribution_test = np.bincount(y_test)
class_distribution_train = np.bincount(y_train)
print("Class distribution in y_test:", class_distribution_test)
print("Class distribution in y_train:", class_distribution_train)

X_train = (X_train * 255).astype(np.uint8)
X_test = (X_test * 255).astype(np.uint8)

# Define a list of augmentation techniques
augmentation = iaa.Fliplr(0.5),  # Horizontal flip with 50% probability

base_classifier = DecisionTreeClassifier(max_depth=1)
adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)
augmentation_reports = {}
unique_classes_test = np.unique(y_test)
num_iterations = 10  # Number of iterations to test each augmentation
consolidated_x = []
consolidated_y = []

for class_label in np.unique(y_train):
    print(f"Class: {class_label}")
    class_mask = (y_train == class_label)
    other_mask = (y_train != class_label)
    augmentation_reports[f'Class_{class_label}'] = {}
    augmented_X_train = X_train[class_mask].copy()
    non_augmented_X_train = X_train[other_mask].copy()
    augmented_X_train = augmentation.augment_images(augmented_X_train)
    all_data = np.concatenate((augmented_X_train, non_augmented_X_train), axis=0)
    all_labels = np.concatenate((y_train[class_mask], y_train[other_mask]), axis=0)
    adaboost_classifier.fit(all_data, all_labels)
    y_pred = adaboost_classifier.predict(X_test)
    print(f'Classes test : {y_test}')
    print(f'Classes pred : {y_pred}')
    classification_rep = classification_report(y_test, y_pred, target_names=unique_classes_test, output_dict=True)
    augmentation_reports[f'Class_{class_label}'] = classification_rep

# Print the classification report for each augmentation technique and class
for class_report, augmentation in augmentation_reports.items():
    print("#" * 40)
    print(f"\Class:  == {class_report} ==\n")
    print("=" * 40)
    for augment, report in augmentation.items():
        print(f"Augmentation {augment} Classification Report:")
        for metric, value in report.items():
            if metric != 'accuracy':
                # print(f"metric: {metric}, value: {value}")
                print(f"{metric}: {value:.2f}" if isinstance(value, (float, np.float32)) else f"{metric}: {value}")
        print("=" * 40)
    print("#" * 40)

with open('metrics_100_per_class.txt', 'w+')as metrics:
    metrics.write(str(augmentation_reports))
